In [1]:
import numpy as np
import pandas as pd 
from astroquery.jplhorizons import Horizons
from astropy import constants

Partiendo de la posición y velocidad de Júpiter el día, mes y año de su cumpleaños a la media noche, calcular, usando el método de Euler y el método de Verlet, la posición y velocidad de Júpiter 10 día después. Compare el resultado con el valor obtenido con Astroquery para ese mismo día.  No cambie las unidades de au y días a metros y segundos y para el parámetro gravitacional del Sol use el valor 0.0002959122 au³/d²

In [18]:
#!pip install -q astroquery
#Importando las librerias necesarias
import numpy as np
import pandas as pd 
from astroquery.jplhorizons import Horizons
from astropy import constants

U_sun=0.0002959122 #au³/d² 
#Teniendo en cuenta que mi fecha de nacimiento es 2002-07-26, entonces extraigamos las condiciones inciales para Jupiter este día:
Condiciones_Jupiter=Horizons(id='599', location='0', epochs={'start': '2002-07-26', 'stop': '2002-08-06', 'step': '10d'}).vectors().to_pandas()[['datetime_str', 'x', 'y', 'z', 'vx', 'vy', 'vz']]

#La primera fila de nuestro dataframe corresponde a las condiciones iniciales de Jupiter en el día de mi nacimiento, y la segunda las condiciones 10 días después:
#Vamos a calcular estas condiciones 10 días después, por el metodo de Euler y Verlet:

#Método de Euler:

#Definamos la función que nos permite encontrar la posición por el método euler:
xf=lambda xi,vi,dt: xi+vi*dt

#Ahora definamos la funcón que nos permite encontrar la velocidad por el método euler:
def vf(vi,xi,r,dt):
    #En este caso la aceleración es generada por la fuerza gravitacional del sol entonces:
    ai=-xi*U_sun/r**3
    return vi+ai*dt

r=np.sqrt(Condiciones_Jupiter.iloc[0][1]**2+Condiciones_Jupiter.iloc[0][2]**2+Condiciones_Jupiter.iloc[0][3]**2)
Condiciones_finales_euler=[]
#Condiciones finales para la posicón 
for i in range(1,4): Condiciones_finales_euler.append(xf(Condiciones_Jupiter.iloc[0][i],Condiciones_Jupiter.iloc[0][i+3],10))
#Condiciones finales para la velocidad
for i in range(1,4): Condiciones_finales_euler.append(vf(Condiciones_Jupiter.iloc[0][i+3],Condiciones_Jupiter.iloc[0][i],r,10))


#Ahora por el método de Verlet:

#Para el caso de Verlet podemos usar las mismas funciones que en el caso de euler, solamente que en el caso de la posicion ahora la velocidad usada
# es la velocidad calculada para las caracteristicas finales
Condiciones_finales_verlet=[]
#Condiciones finales para la posicón
for i in range(1,4): Condiciones_finales_verlet.append(xf(Condiciones_Jupiter.iloc[0][i],Condiciones_finales_euler[i+2],10))

#Uniendo las codiciones finales por el metodo de verlet 
Condiciones_finales_verlet=Condiciones_finales_verlet+[Condiciones_finales_euler[i] for i in range(3,6)]

# Extrayendo los valores reales
valores_reales = Condiciones_Jupiter.iloc[1]

# Imprimiendo los resultados
print("Fecha inicial:", Condiciones_Jupiter.loc[0, 'datetime_str'])
print("Fecha  Final:", Condiciones_Jupiter.loc[1, 'datetime_str'])

print("\nValores reales después de 10 días, seguún Astroquery:")
print(f"x: {valores_reales['x']}, y: {valores_reales['y']}, z: {valores_reales['z']}")
print(f"vx: {valores_reales['vx']}, vy: {valores_reales['vy']}, vz: {valores_reales['vz']}")

print("\nCondiciones estimadas por el método de Euler después de 10 días:")
print(f"x: {Condiciones_finales_euler[0]}, y: {Condiciones_finales_euler[1]}, z: {Condiciones_finales_euler[2]}")
print(f"vx: {Condiciones_finales_euler[3]}, vy: {Condiciones_finales_euler[4]}, vz: {Condiciones_finales_euler[5]}")

print("\nCondiciones estimadas por el método de Verlet después de 10 días:")
print(f"x: {Condiciones_finales_verlet[0]}, y: {Condiciones_finales_verlet[1]}, z: {Condiciones_finales_verlet[2]}")
print(f"vx: {Condiciones_finales_verlet[3]}, vy: {Condiciones_finales_verlet[4]}, vz: {Condiciones_finales_verlet[5]}")

#Ahora calculemos el error porcentual para cada una de las condiciones finales, con respecto a los valores reales obtenidos por Astroquery:
# Calculando los errores porcentuales para el método de Euler
errores_euler = [abs((Condiciones_finales_euler[i] - valores_reales[i+1]) / valores_reales[i+1]) * 100 for i in range(6)]

# Calculando los errores porcentuales para el método de Verlet
errores_verlet = [abs((Condiciones_finales_verlet[i] - valores_reales[i+1]) / valores_reales[i+1]) * 100 for i in range(6)]

# Imprimiendo los errores porcentuales
print("\nErrores porcentuales por el método de Euler:")
print(f"x: {errores_euler[0]:.3f}%, y: {errores_euler[1]:.3f}%, z: {errores_euler[2]:.3f}%")
print(f"vx: {errores_euler[3]:.3f}%, vy: {errores_euler[4]:.3f}%, vz: {errores_euler[5]:.3f}%")

print("\nErrores porcentuales por el método de Verlet:")
print(f"x: {errores_verlet[0]:.3f}%, y: {errores_verlet[1]:.3f}%, z: {errores_verlet[2]:.3f}%")
print(f"vx: {errores_verlet[3]:.3f}%, vy: {errores_verlet[4]:.3f}%, vz: {errores_verlet[5]:.3f}%")

print(f'\nComo se puede observar en los porcentajes de error el método de verlet en este caso fue un poco más preciso')


Fecha inicial: A.D. 2002-Jul-26 00:00:00.0000
Fecha  Final: A.D. 2002-Aug-05 00:00:00.0000

Valores reales después de 10 días, seguún Astroquery:
x: -3.185579180768156, y: 5.364921313548042, z: 0.03677344745148706
vx: -0.01929145383012097, vy: -0.01488380355524235, vz: 0.0002103996277248161

Condiciones estimadas por el método de Euler después de 10 días:
x: -3.1955875942999756, y: 5.376169199026981, z: 0.03713618019279433
vx: -0.02106202479736272, vy: -0.012554913474460883, vz: 0.00020087056402323336

Condiciones estimadas por el método de Verlet después de 10 días:
x: -3.195227341106062, y: 5.375505201276991, z: 0.03713194068748877
vx: -0.02106202479736272, vy: -0.012554913474460883, vz: 0.00020087056402323336

Errores porcentuales por el método de Euler:
x: 0.314%, y: 0.210%, z: 0.986%
vx: 9.178%, vy: 15.647%, vz: 4.529%

Errores porcentuales por el método de Verlet:
x: 0.303%, y: 0.197%, z: 0.975%
vx: 9.178%, vy: 15.647%, vz: 4.529%

Como se puede observar en los porcentajes de err

In [26]:
Condiciones_Jupiter

,datetime_str,x,y,z,vx,vy,vz
0,A.D. 2002-Jul-26 00:00:00.0000,-2.984607,5.501054,0.035123,-0.021098,-0.012489,0.000201
1,A.D. 2002-Aug-05 00:00:00.0000,-3.185579,5.364921,0.036773,-0.019291,-0.014884,0.000210


In [24]:
((-2.984607)**2+5.501054**2+0.035123**2)**0.5

6.2586506277706535